## Introduction to DataBase System

데이터베이스(Database)는 데이터를 체계적으로 저장하고 관리하는 시스템이다. 데이터를 구조화된 형태로 저장하여 효율적인 검색, 수정, 삭제가 가능하도록 돕는다. 데이터베이스는 크게 엑셀처럼 행과 열로 관리되는 관계형 데이터베이스(RDBMS)와 키와 밸류 형태로 관리되는 NoSQL 데이터베이스로 이뤄진다. 

그중 관계형 데이터베이스는 데이터를 테이블 형태로 저장하며, 각 테이블은 행(Row)과 열(Column)로 구성된다. 데이터베이스는 대량의 데이터를 안정적으로 관리하고, 여러 사용자와 애플리케이션이 동시에 데이터를 활용할 수 있게 한다. 관계형 데이터베이스는 특히 금융 데이터를 다루는 데 있어서 알맞은 형태로, 시계열 및 고객 데이터와 거래 기록들을 다루는 데 있어서 알맞게 사용가능하다. 최근에는 High Frequency 데이터 처리 및 market depth, order book 데이터 관리의 중요성이 대두되면서, NoSQL형 데이터도 증가하는 추세에 있다.

In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3

TICKERS = ['^SPX','^IXIC','^RUT','^DJI','^XAX']
START_DATE = '1990-01-01'

### 1. Structured Query Language

`SQL(Structured Query Language)`은 관계형 데이터베이스를 관리하기 위한 표준 언어이다. 1970년대 IBM에서 SEQUEL이라는 이름으로 처음 개발되었으며, 이후 ANSI와 ISO에 의해 표준화되었다. SQL은 관계형 데이터베이스에서 데이터를 표 형식으로 다루며, 데이터 간의 관계를 정의하고 관리하는 데 적합하다. 

SQL은 데이터 정의, 조작, 제어라는 세 가지 주요 기능으로 나뉘는데, **데이터 정의** 는 테이블과 같은 데이터베이스 구조를 생성, 수정, 삭제하는 작업으로, 주로 데이터베이스 설계 초기 단계에서 사용된다. **데이터 조작** 은 삽입, 조회, 수정, 삭제와 같은 데이터를 직접 다루는 작업으로, 데이터베이스의 일상적인 사용에서 가장 많이 쓰인다. **데이터 제어** 는 사용자 권한을 관리하고 데이터베이스의 보안을 설정하는 데 초점이 맞춰져 있다. 이러한 기능은 데이터베이스의 설계부터 운영까지 모든 단계에서 필수적이다. 예를 들어, 새로운 테이블을 생성하고 데이터를 삽입한 뒤, 특정 사용자에게만 접근 권한을 부여하는 방식으로 활용될 수 있다. SQL의 각 기능은 효율적인 데이터 관리를 가능하게 하며, 데이터 무결성과 보안을 보장한다.

SQL은 데이터를 정의(데이터의 구조를 정의하고 DB를 생성)하는 DDL(Data Definition Language), 데이터를 조작(수정, 삭제 등)하는 DML(Data Manipulation Language), 데이터를 제어(DB 접근권한 설정 등)하는 DCL(Data Control Language)로 구성되어 있다. 이 세 가지 기능은 데이터베이스의 구조 설계, 데이터 처리, 보안 관리의 모든 단계를 포함한다. 

현재 MySQL, PostgreSQL, SQLite와 같은 많은 데이터베이스 시스템이 SQL을 지원하며, 효율적이고 안정적인 데이터 관리를 가능하게 한다.

#### 1.1 DDL (Data Definition Language)

테이블과 관련 열을 생성, 변경, 삭제하는 명령어를 데이터 정의어라고 한다. DDL은 데이터베이스의 구조를 정의하고 관리하는 데 사용된다. 테이블, 인덱스, 뷰 등의 데이터베이스 객체를 생성, 수정, 삭제하는 명령어를 포함한다. 대표적인 명령어로는 `CREATE`, `ALTER`, `DROP` 등이 있다. 

예를 들어, `CREATE TABLE` 명령어를 사용해 새로운 테이블을 정의할 수 있다. 또한, `ALTER TABLE`을 사용해 기존 테이블에 새로운 열을 추가하거나 수정할 수 있다. 

DDL은 데이터베이스의 설계 초기 단계에서 중요하며, 데이터의 논리적 구조(어떤 type의 데이터가 어디에 들어가고, 어디에 연결될지 등등을 정의해야 함)를 설정하는 역할을 한다. 데이터 타입과 제약 조건을 설정함으로써 데이터의 일관성과 무결성을 보장할 수 있다. 금융 데이터베이스에서는 주식 가격, 거래 기록, 경제 지표 등을 저장하는 테이블을 설계하는 데 자주 사용된다. 스키마(데이터의 구조와 구성을 정의하는 설계도를 의미) 설계와 데이터 정의는 데이터베이스 성능과 데이터 분석의 정확성에 큰 영향을 미친다.

**Create Table**

가장 기초적인 작업은 테이블을 생성하는 것이다. 테이블은 하나의 엑셀 데이터를 생성하는 것과 같다고 보면 된다. 한 종목의 시계열 데이터는 대략 7개의 컬럼(날짜, 시가, 고가, 저가, 종가, 수정종가, 거래량)을 가진 table을 생성한다고 보면 편하다. DB데이터는 `sqlite3` 라이브러리의 `connect()` 함수를 통해 연결을 우선적으로 해 줘야 한다. `cursur()` method는 파이썬 내에서 SQL 명령문을 실행하도록 해 주는 역할을 한다.

In [3]:
DB_PATH = 'price.db'
conn = sqlite3.connect(DB_PATH)  # DB에 연결
cursor = conn.cursor()   # DB에 접근해 SQL을 실행가능한 상태로 변경

In [4]:
type(conn) # Connection 

sqlite3.Connection

price.db에 `stock_prices`이라는 table을 생성해 보자

`CREATE table if not exists stock_prices`는 'stock_price'라는 테이블이 존재하지 없으면 테이블을 만드라는 의미
SQL문에서 column을 만들 때, 그 column에 들어갈 데이터 형태도 지정해 줘야 함

In [5]:
cursor.execute(
    f"""
        CREATE table if not exists stock_prices (
            price varchar(10),
            ticker varchar(10),
            value real,
            primary key (price, ticker)
        )
    """
)

DB에 접속하면 실제로 stock_prices이라는 이름의 table이 생성되는 것을 확인할 수 있다. 여기서 SQL문의 `primary_key`는 테이블에서 각 행을 유일하게 식별하는 컬럼으로, 데이터를 불러서 읽을 때 좀 더 빠르게 읽을 수 있도록 한다. 테이블과 열의 이름을 정할 때에는 다음 규칙을 따라야 한다.

- 동일 한 사용자가 동일한 이름의 테이블을 중복하여 생성이 불가능하다. 또한, 테이블 안의 열 이름도 동일한 이름으로 생성 불가능하다.
- `SELECT`, `FROM`, `COUNT`와 같은 예약어는 사용할 수 없다.
- 테이블 이름은 문자로 시작해야 한다. 이 규칙은 python의 변수 사용 규칙과 같다. 한글과 특문을 사용할 수는 있지만 권장되지 않는다.
- 문자의 길이는 1~30 byte 이내여야 한다.

**Modify Tables**

기존에 존재하는 테이블에 컬럼을 추가하여 저장하거나, 기존의 컬럼의 내용을 수정하고 싶을 수 있다. `ALTER` 명령문은 SQL에서 테이블 또는 데이터베이스 객체의 구조를 변경할 때 사용된다. 테이블에 새로운 열(Column)을 추가하거나 기존 열을 수정 또는 삭제하는 등의 작업을 수행할 수 있다. 또한, 제약 조건(Constraints) 추가, 데이터 타입 변경 등도 가능하다.

이미 생성한 테이블에 맨 끝쪽에 returns를 추가한다고 상상해 보자. 아래와 같이 SQL문을 작성할 수 있다.

In [6]:
cursor.execute(
    f"""
        ALTER TABLE stock_prices
        add returns real
    """
)

또는, 이미 추가된 컬럼을 삭제할 수도 있다.

In [7]:
cursor.execute(
    f"""
        ALTER TABLE stock_prices
        drop column returns;
    """
)

`ALTER TABLE ~ ADD`절의 특징은 다음과 같다.

- 새로운 열을 추가할 수 있지만, 테이블에 있던 기존의 열은 삭제할 수 없다.
- 새로 생성되는 열은 위치를 지정할 수 없다
- 테이블에 이미 행이 있다면 열을 추가하였을 때 새로운 열의 데이터 값을 `NULL`로 초기화된다.

`ALTER TABLE ~ MODIFY`절을 사용하여 열의 정의를 수정할 수 있다. 열의 수정 대상은 열의 데이터 타입과 크기, Default Value이다. 단, 여기서는 SQLlite3을 사용하기 때문에, Modify는 지원되지 않는다.

`ALTER TABLE ~ RENAME COLUMN` 절을 사용하면 열 이름을 수정할 수 있다.

In [8]:
cursor.execute(
    f"""
        ALTER TABLE stock_prices
        add returns real
    """
)

In [9]:
cursor.execute(
    f"""
        alter table stock_prices
        rename column returns to return;
    """
)

**Delete table**

table을 생성한 후 더이상 사용하지 않거나 쓸모 없다고 판단되는 경우 table 자체를 삭제할 수도 있다. `DROP` 명령문은 데이터베이스에서 특정 테이블을 완전히 삭제하는 데 사용된다. 이 명령어를 실행하면 테이블의 구조와 모든 데이터가 완전히 제거되며, 복구할 수 없다. 삭제된 테이블과 관련된 모든 인덱스, 제약 조건, 그리고 해당 테이블에 저장된 데이터도 함께 삭제된다.

In [10]:
cursor.execute(
    f"""
        DROP table stock_prices
    """
)

이제 실제로 테이블을 생성해 보자. 여기서 `NOT NULL`은 해당 column에는 null 값을 허용하지 않는다는 의미이다.

In [11]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS stock_prices (
        date DATE NOT NULL,
        field VARCHAR(10) NOT NULL,
        ticker VARCHAR(10) NOT NULL,
        value REAL NOT NULL,
        PRIMARY KEY (date, field, ticker)
    )
""")

#### 1.2 DML (Data Manipulation Language)

테이블을 생성하였으니, 알맞은 데이터를 적재할 차례이다. DML은 데이터베이스에 저장된 데이터를 조회하거나 수정, 삽입, 삭제하는 작업을 수행한다. **가장 자주 사용되는 SQL 기능**으로, 데이터를 직접 다루기 때문에 **실시간 데이터 분석과 관리에서 중요**하다. 

대표적인 명령어로는 데이터를 삽입하는 `INSERT`, 데이터를 검색하는 `SELECT`, 데이터를 수정하는 `UPDATE`, 데이터를 삭제하는 `DELETE`가 있다. 예를 들어, `SELECT` 명령어를 사용해 특정 주식의 일별 가격을 조회할 수 있다. 혹은, 전체 가격 테이블에서 특정 종목의 가격 전체의 열을 불러올 수도 있다.

주로 데이터의 상태를 변경하거나 원하는 데이터를 추출하는 데 사용되며, 데이터베이스 사용자의 주요 인터페이스 역할을 한다. 금융 데이터에서는 특정 날짜의 주식 가격을 조회하거나, 거래 내역을 삽입하고 수정하는 데 활용된다. DML은 데이터를 효율적으로 다룰 수 있도록 설계되어 있어, 대규모 데이터베이스에서도 빠르고 정확하게 작동한다.

In [12]:
data = yf.download(TICKERS, start = START_DATE, progress = False)

YF.download() has changed argument auto_adjust default to True


yahoo finance를 비롯하여 대부분의 가격을 다루는 DataBase는 다음과 같이 MultiIndex 형태로 스키마를 관리한다.

In [13]:
data.columns

MultiIndex([( 'Close',  '^DJI'),
            ( 'Close', '^IXIC'),
            ( 'Close',  '^RUT'),
            ( 'Close',  '^SPX'),
            ( 'Close',  '^XAX'),
            (  'High',  '^DJI'),
            (  'High', '^IXIC'),
            (  'High',  '^RUT'),
            (  'High',  '^SPX'),
            (  'High',  '^XAX'),
            (   'Low',  '^DJI'),
            (   'Low', '^IXIC'),
            (   'Low',  '^RUT'),
            (   'Low',  '^SPX'),
            (   'Low',  '^XAX'),
            (  'Open',  '^DJI'),
            (  'Open', '^IXIC'),
            (  'Open',  '^RUT'),
            (  'Open',  '^SPX'),
            (  'Open',  '^XAX'),
            ('Volume',  '^DJI'),
            ('Volume', '^IXIC'),
            ('Volume',  '^RUT'),
            ('Volume',  '^SPX'),
            ('Volume',  '^XAX')],
           names=['Price', 'Ticker'])

SQL의 `INSERT`를 활용해 개별적으로 데이터를 적재할 수 있다. 이러는 경우, 모든 데이터에 대해서 하나하나 데이터를 적재하면 된다

In [14]:
data = data.stack(level=1).reset_index()  # multicolumn 데이터를 singlecolumn 데이터로 변환 가능
data

C:\Users\kumb2\AppData\Local\Temp\ipykernel_28780\428656077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(level=1).reset_index()  # multicolumn 데이터를 singlecolumn 데이터로 변환 가능


Price,Date,Ticker,Close,High,Low,Open,Volume
0,1990-01-02,^IXIC,459.299988,459.299988,452.700012,452.899994,1.107200e+08
1,1990-01-02,^RUT,169.949997,169.960007,168.220001,168.309998,1.620700e+08
2,1990-01-02,^SPX,359.690002,359.690002,351.980011,353.399994,1.620700e+08
3,1990-01-03,^IXIC,460.899994,461.600006,460.000000,461.100006,1.526600e+08
4,1990-01-03,^RUT,170.789993,170.800003,169.960007,169.960007,1.923300e+08
...,...,...,...,...,...,...,...
42300,2025-03-13,^DJI,40813.570312,41360.121094,40661.769531,41280.050781,6.467700e+08
42301,2025-03-13,^IXIC,17303.009766,17621.910156,17239.439453,17598.560547,7.147050e+09
42302,2025-03-13,^RUT,1993.689941,2031.160034,1984.920044,2026.079956,5.018980e+09
42303,2025-03-13,^SPX,5521.520020,5597.779785,5504.649902,5594.450195,5.018980e+09


In [15]:
data = data[['Date','Ticker','Open','High','Low','Close','Volume']]

In [16]:
data.columns = ['date', 'ticker', 'Open', 'High', 'Low', 'Close', 'Volume']

In [17]:
data['date'] = data['date'].astype(str) # sql lite는 datetime 형태의 타입이 지원되지 않으므로, str형태로 변환

In [ ]:
records = []
for _, row in data.iterrows():
    for field in ['Open', 'High', 'Low', 'Close', 'Volume']:
        records.append((row['date'], field, row['ticker'], row[field]))

In [ ]:
cursor.executemany(
    "insert or ignore into stock_prices (date, field, ticker, value) values (?, ?, ?, ?)", 
    records
)

In [ ]:
conn.commit() # commit 실행 => 모든 작업이 정상적으로 처리되었음을 알려주고 처리 내용을 모두 저장하는 메소드

In [ ]:
df = pd.read_sql("SELECT * FROM stock_prices LIMIT 10", conn) # 저장된 데이터를 확인
print(df)

         date   field ticker         value
0  1990-01-02    Open  ^IXIC  4.529000e+02
1  1990-01-02    High  ^IXIC  4.593000e+02
2  1990-01-02     Low  ^IXIC  4.527000e+02
3  1990-01-02   Close  ^IXIC  4.593000e+02
4  1990-01-02  Volume  ^IXIC  1.107200e+08
5  1990-01-02    Open   ^RUT  1.683100e+02
6  1990-01-02    High   ^RUT  1.699600e+02
7  1990-01-02     Low   ^RUT  1.682200e+02
8  1990-01-02   Close   ^RUT  1.699500e+02
9  1990-01-02  Volume   ^RUT  1.620700e+08


In [ ]:
conn.close() # 데이터 연결 종료

#### 1.3 DCL (Data Control Language)

DCL은 데이터베이스에 대한 접근 권한을 관리하고 보안을 유지하는 데 사용된다. 대표적인 명령어로는 권한을 부여하는 `GRANT`와 권한을 회수하는 `REVOKE`가 있다. 예를 들어, 특정 사용자에게 주식 데이터베이스의 읽기 권한만 부여하거나, 수정 권한을 제한할 수 있다. 

DCL은 데이터베이스의 무단 접근을 방지하고, 데이터의 기밀성을 보장하는 데 중요한 역할을 한다. 금융 데이터베이스에서는 민감한 데이터에 대한 접근을 통제해 데이터 유출이나 보안 사고를 예방할 수 있다. 또한, 데이터베이스 관리자는 사용자의 역할(Role)을 설정하고, 각 역할에 적합한 권한을 할당할 수 있다. 따라서 DCL은 데이터 무결성과 신뢰성을 유지하는 데 필수적이다. 데이터 보호가 중요한 환경에서는 DCL 명령어를 활용한 권한 관리를 정기적으로 점검해야 한다.

여기서는 데이터 적재 및 관리와 거리가 멀기 때문에 따로 다루지는 않도록 한다.

### 2. Rules of Data Management

Data Base에서 무결성과 제약 조건은 반드시 지켜져야 하는 약속이다. 무결성과 제약 조건은 무엇인지 알아보자.

#### 2.1 Data Integrity

데이터베이스에 저장된 데이터 값과 사용자가 의도한 데이터 값은 일치헤야 한다. 또한, 데이터는 정확성, 유효성, 일관성, 신뢰성이 지켜져야 하며, 이를 어기는 잘못된 입력과 갱신 또는 삭제로부터 보호되어야 한다. 예를 들어, 특정 종목의 코드 또는 Ticker는 식별자이기 때문에 identification과 같으므로 유일한 값이어야 한다. 그러나, key 값에 중복 값이 입력되거나 `NULL`값이 입력된다면 데이터 식별에 큰 문제가 생길 수 있다. 둘째로, 숫자로 이뤄진 컬럼에 'two'와 같은 문자열이 들어가는 경우 이 값을 숫자로 인정해야 할지 말아야 할지 어렵게 된다.

이처럼, 데이터베이스의 관리 규칙이 지켜지지 않으면 데이터베이스 시스템을 운영하는 데 큰 혼란이 올 수 있다. 데이터베이스의 데이터는 사용자의 목적에 맞게 입력되고 저장되어야 하며, **규칙을 위반하지 말아야** 한다. 이를 위해 DB system은 데이터에 접근하거나 데이터를 처리할 때마다 부적절한 데이터가 입력되는지 검사하여 데이터가 결점이 없는 것을 검사한다. 이 기본 규칙을 Data Integrity라고 한다.

|유형| 설명                                                                                                                                       |
|:---:|:-----------------------------------------------------------------------------------------------------------------------------------------|
|Entity Integrity| primary key로 선택된 열은 고유해야 하며, NULL값을 가질 수 없다.                                                                                             |
|Reference Integrity| primary key와 foreign key의 관계이다. <br> 외래 키가 있는 테이블의 경우, 기본 키와 외래 키 간의 관계가 항상 유지됨을 보장한다. <br> 참조하는 외래 키가 존재하면 행은 삭제될 수 없고, 기본 키도 변경될 수 없다. |
|Domain Integrity| 데이터 형태, 범위, 기본값, 유일성에 관한 제한이다. <br> 주어진 속상 값은 그 속성이 정의된 도메인에 속한 값이어야 한다.                                                                 |
|Business Integrity| 사용자의 업무 규칙에 따른 비즈니스적인 제약 조건이다                                                                                                            |

#### 2.2 Constraint 

제약 조건이란, 정해 놓은 규칙에 맞는 데이터만 입력받고 규칙에 어긋나는 데이터는 거부하여 데이터 무결성을 지키는 방법이다. 대표적인 제약 조건의 종류는 다음과 같다.

##### 1. Primary Key Constraint

`UNIQUE` + `NOT NULL`을 만족해야 한다. 또한, 테이블을 대표하여 각 행을 유일하게 식별하는 값이어야 한다.

In [ ]:
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

In [ ]:
cursor.execute("PRAGMA table_info(stock_prices);") # table 내의 primary key 가져오기
columns = cursor.fetchall()

In [ ]:
columns

[(0, 'date', 'DATE', 1, None, 1),
 (1, 'field', 'VARCHAR(10)', 1, None, 2),
 (2, 'ticker', 'VARCHAR(10)', 1, None, 3),
 (3, 'value', 'REAL', 1, None, 0)]

##### 2. Foreign Key Constraint

열 값이 부모 테이블의 참조 열을 반드시 참조해야 한다. 참조되는 열은 `UNIQUE`하거나 `PRIMARY KEY`이다

In [ ]:
cursor.execute(
    """
    create table if not exists ticker_info (
        number integer primary key autoincrement, 
        ticker varchar(10) not null
    )
    """
)

In [ ]:
cursor.executemany(
    "insert or ignore into ticker_info (ticker) values (?)", 
    [(ticker,) for ticker in TICKERS]
)
conn.commit()

In [ ]:
cursor.execute("SELECT * FROM ticker_info;")
print(cursor.fetchall())

[(1, '^SPX'), (2, '^IXIC'), (3, '^RUT'), (4, '^DJI'), (5, '^XAX')]


In [ ]:
cursor.execute("SELECT * FROM ticker_info;")
ticker_info = cursor.fetchall()

In [ ]:
ticker_info

[(1, '^SPX'), (2, '^IXIC'), (3, '^RUT'), (4, '^DJI'), (5, '^XAX')]

`join` keyword를 사용하면 ticker info로부터 저장된 티커를 참조하여 stock prices의 value를 가져올 수 있다.
`where`문은 조건을 추가해주는 역할을 함

In [ ]:
cursor.execute(
    """
    select stock_prices.date, stock_prices.field, stock_prices.ticker, stock_prices.value
    from stock_prices
    join ticker_info ON stock_prices.ticker = ticker_info.ticker
    where stock_prices.ticker = '^SPX'
    """
)

In [ ]:
pd.DataFrame(cursor.fetchall())

,0,1,2,3
0,1990-01-02,Open,^SPX,3.534000e+02
1,1990-01-02,High,^SPX,3.596900e+02
2,1990-01-02,Low,^SPX,3.519800e+02
3,1990-01-02,Close,^SPX,3.596900e+02
4,1990-01-02,Volume,^SPX,1.620700e+08
...,...,...,...,...
44320,2025-03-13,Open,^SPX,5.594450e+03
44321,2025-03-13,High,^SPX,5.597780e+03
44322,2025-03-13,Low,^SPX,5.504650e+03
44323,2025-03-13,Close,^SPX,5.521520e+03


`where` 조건을 두 개 이상 사용해 특정 티커의 수정 종가만 불러올 수 있다.

`JOIN`은 판다스의 JOIN 메소드와 비슷하다. on은 join의 조건을 의미하며, where을 이용해 데이터 검색 조건을 추가할 수 있다.

In [ ]:
cursor.execute(
    """
    select stock_prices.date, stock_prices.field, stock_prices.ticker, stock_prices.value
    from stock_prices
    join ticker_info ON stock_prices.ticker = ticker_info.ticker
    where stock_prices.ticker = '^SPX' and stock_prices.field = 'Adj Close'
    """
)

In [ ]:
adj_close = pd.DataFrame(cursor.fetchall())

In [ ]:
adj_close.head()

""


##### 3. Unique Key Constraint

중복된 값을 허용하지 않는다. 유일한 값으로 존재해야 한다.

##### 4. Not NULL Constraint

`NULL` 값을 허용하지 않는다. 반드시 값을 입력해야 한다.

In [ ]:
cursor.execute(
    """
    SELECT * FROM stock_prices WHERE date = '2025-02-14'
    """
)
cursor.fetchall()

[('2025-02-14', 'Close', '^DJI', 44546.078125),
 ('2025-02-14', 'Close', '^IXIC', 20026.76953125),
 ('2025-02-14', 'Close', '^RUT', 2279.97998046875),
 ('2025-02-14', 'Close', '^SPX', 6114.6298828125),
 ('2025-02-14', 'Close', '^XAX', 5100.7099609375),
 ('2025-02-14', 'High', '^DJI', 44769.05078125),
 ('2025-02-14', 'High', '^IXIC', 20045.759765625),
 ('2025-02-14', 'High', '^RUT', 2297.7099609375),
 ('2025-02-14', 'High', '^SPX', 6127.47021484375),
 ('2025-02-14', 'High', '^XAX', 5204.830078125),
 ('2025-02-14', 'Low', '^DJI', 44498.9609375),
 ('2025-02-14', 'Low', '^IXIC', 19932.150390625),
 ('2025-02-14', 'Low', '^RUT', 2276.6298828125),
 ('2025-02-14', 'Low', '^SPX', 6107.6201171875),
 ('2025-02-14', 'Low', '^XAX', 5088.31005859375),
 ('2025-02-14', 'Open', '^DJI', 44720.98828125),
 ('2025-02-14', 'Open', '^IXIC', 19956.8203125),
 ('2025-02-14', 'Open', '^RUT', 2287.85009765625),
 ('2025-02-14', 'Open', '^SPX', 6115.52001953125),
 ('2025-02-14', 'Open', '^XAX', 5140.9501953125),
 (

In [ ]:
conn.close()  # 요거 안해서 Isolation에 걸려서 Lock이 걸려버리는 경우가 존재!

#### Exercise 1

관심있는 종목 데이터를 수집하고 수익률을 계산한 뒤 데이터와 수익률을 db에 저장해봅시다

In [1]:
TICKERS = ['^SPX','^IXIC','^RUT','^DJI','^XAX']
START_DATE = '1990-01-01'

In [2]:
import yfinance as yf
df= yf.download(
    TICKERS,
    start = START_DATE,
    end = '2025-02-22',
    interval = '1d', # 데이터 추출 빈도 (기본값 : 1일)
    progress = False, # 다운로드 시 progress bar 표시 여부
    auto_adjust = True, 
    multi_level_index = False # Multi Index형태의 컬럼 지정 해제
)

In [3]:
import re

df.dropna(inplace=True)
df=df.squeeze(axis=1)
df_Close=df.Close

TICKERS = df.Close.columns
cleaned_tickers = [re.sub(r'[^a-zA-Z]', '', ticker) for ticker in TICKERS]
df_Close.columns=cleaned_tickers

In [6]:
df_ret=np.log(df_Close)-np.log(df_Close.shift(1))
df_ret=df_ret.dropna()

df_ret.columns=cleaned_tickers

In [7]:
Datas=[]
for ticker in df_Close.columns:
    globals()[str(ticker)]=pd.concat([df_Close[ticker], df_ret[ticker]], axis=1).dropna()
    globals()[str(ticker)].columns = [ticker, 'Return']
    Datas.append(globals()[str(ticker)])

Datas[0]

,DJI,Return
Date,,
1995-12-28,5095.799805,-0.001984
1995-12-29,5117.120117,0.004175
1996-01-02,5177.450195,0.011721
1996-01-03,5194.069824,0.003205
1996-01-04,5173.839844,-0.003902
...,...,...
2025-02-14,44546.078125,-0.003705
2025-02-18,44556.339844,0.000230
2025-02-19,44627.589844,0.001598


In [8]:
DB_PATH = 'price_data.db'
conn = sqlite3.connect(DB_PATH)  # DB에 연결
cursor = conn.cursor()   # DB에 접근해 SQL을 실행가능한 상태로 변경
tickers=cleaned_tickers

for ticker in tickers:
    query=f"""
            CREATE table if not exists {ticker} (
                date varchar(15),
                price real,
                return real,
                primary key (date)
            )
        """
    cursor.execute(query)


In [9]:
tickers

['DJI', 'IXIC', 'RUT', 'SPX', 'XAX']

In [10]:
# '^SPX','^IXIC','^RUT'
for Data in Datas:
    records = []
    ticker=Data.columns[0]
    print(ticker)
    for date, row in Data.iterrows():
        records.append((str(date)[:-9], row.iloc[0], row.iloc[1]))
    
    cursor.executemany(f"""insert or ignore into {ticker} (date, price, return) values (?, ?, ?)""", records)

conn.commit()

DJI
IXIC
RUT
SPX
XAX


#### Exercise 2

db에서 다시 데이터를 받아와봅시다

In [12]:
DB_PATH = 'price_data.db'
conn = sqlite3.connect(DB_PATH)  # DB에 연결
cursor = conn.cursor()   # DB에 접근해 SQL을 실행가능한 상태로 변경

In [21]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'") # 저장된 데이터를 확인 (sqlite_master를 사용하면 테이블 이름 조회 가능)
tables = cursor.fetchall()  # 행 데이터를 다 받아오는 코드
tables=[table[0] for table in tables]
tables

['IXIC', 'RUT', 'SPX', 'DJI', 'XAX']

In [ ]:
# IXIC 데이터만 가져옴
df = pd.read_sql(f"SELECT * FROM {tables[0]}", conn, index_col='date')
df

,price,return
date,,
1990-01-03,460.899994,0.003478
1990-01-04,459.399994,-0.003260
1990-01-05,458.200012,-0.002615
1990-01-08,458.700012,0.001091
1990-01-09,456.799988,-0.004151
...,...,...
2025-02-14,20026.769531,0.004059
2025-02-18,20041.259766,0.000723
2025-02-19,20056.250000,0.000748


In [29]:
# XAX 수익률만 가져옴
df = pd.read_sql(f"SELECT date, return FROM {tables[4]}", conn, index_col='date')
df

,return
date,
1995-12-28,-0.000403
1995-12-29,0.006970
1996-01-02,0.005657
1996-01-03,0.002221
1996-01-04,-0.006806
...,...
2025-02-14,-0.007858
2025-02-18,0.014365
2025-02-19,0.001423


In [11]:
conn.close()